## 1. Importar Librerías

In [ ]:
import os
from dotenv import load_dotenv
from typing import Any
from agent_framework import AgentExecutorResponse, WorkflowBuilder
from agent_framework.openai import OpenAIChatClient
from azure.identity import DefaultAzureCredential
from azure.identity.aio import get_bearer_token_provider
from pydantic import BaseModel

## 2. Configuración

In [ ]:
load_dotenv()

client = OpenAIChatClient(
    base_url = os.getenv("AZURE_OPENAI_ENDPOINT") + "/openai/v1/",
    api_key = get_bearer_token_provider(
        DefaultAzureCredential(), 
        "https://cognitiveservices.azure.com/.default"),
    model_id = os.getenv("MODEL"),
)

print(f"✅ Cliente configurado con modelo: {os.getenv('MODEL')}")

## 3. Definir Modelo de Resultado Estructurado

Usamos Pydantic para definir la estructura de respuesta del reviewer. Esto garantiza que siempre recibamos datos con el formato esperado.

In [ ]:
class ReviewResult(BaseModel):
    """Review evaluation with scores and feedback."""
    score: int  # Overall quality score (0-100)
    feedback: str  # Concise, actionable feedback
    clarity: int  # Clarity score (0-100)
    completeness: int  # Completeness score (0-100)
    accuracy: int  # Accuracy score (0-100)
    structure: int  # Structure score (0-100)

print("✅ Modelo ReviewResult definido")

## 4. Funciones de Condición para Ramificación

Estas funciones determinan qué camino toma el workflow basándose en el score de calidad.

In [ ]:
def needs_editing(message: Any) -> bool:
    """Check if content needs editing based on review score."""
    if not isinstance(message, AgentExecutorResponse):
        return False
    try:
        review = ReviewResult.model_validate_json(message.agent_run_response.text)
        print(f"  📊 Score: {review.score} - Needs editing: {review.score < 80}")
        return review.score < 80
    except Exception:
        return False

def is_approved(message: Any) -> bool:
    """Check if content is approved (high quality)."""
    if not isinstance(message, AgentExecutorResponse):
        return True
    try:
        review = ReviewResult.model_validate_json(message.agent_run_response.text)
        print(f"  ✅ Score: {review.score} - Approved: {review.score >= 80}")
        return review.score >= 80
    except Exception:
        return True

print("✅ Funciones de condición definidas")

## 5. Crear Agente Writer

Genera contenido inicial basado en la solicitud del usuario.

In [ ]:
writer = client.create_agent(
    name="Writer",
    instructions=(
        "You are an excellent content writer. "
        "Create clear, engaging content based on the user's request. "
        "Focus on clarity, accuracy, and proper structure."
    ),
)

print("✅ Writer Agent creado")

## 6. Crear Agente Reviewer

Evalúa el contenido y proporciona feedback estructurado usando el modelo `ReviewResult`.

In [ ]:
reviewer = client.create_agent(
    name="Reviewer",
    instructions=(
        "You are an expert content reviewer. "
        "Evaluate the writer's content based on:\n"
        "1. Clarity - Is it easy to understand?\n"
        "2. Completeness - Does it fully address the topic?\n"
        "3. Accuracy - Is the information correct?\n"
        "4. Structure - Is it well-organized?\n\n"
        "Return a JSON object with:\n"
        "- score: overall quality (0-100)\n"
        "- feedback: concise, actionable feedback\n"
        "- clarity, completeness, accuracy, structure: individual scores (0-100)"
    ),
    response_format=ReviewResult,
)

print("✅ Reviewer Agent creado (con formato de respuesta estructurado)")

## 7. Crear Agente Editor

Mejora el contenido basándose en el feedback del reviewer.

In [ ]:
editor = client.create_agent(
    name="Editor",
    instructions=(
        "You are a skilled editor. "
        "You will receive content along with review feedback. "
        "Improve the content by addressing all the issues mentioned in the feedback. "
        "Maintain the original intent while enhancing clarity, completeness, accuracy, and structure."
    ),
)

print("✅ Editor Agent creado")

## 8. Crear Agente Publisher

Formatea el contenido final para publicación.

In [ ]:
publisher = client.create_agent(
    name="Publisher",
    instructions=(
        "You are a publishing agent. "
        "You receive either approved content or edited content. "
        "Format it for publication with proper headings and structure."
    ),
)

print("✅ Publisher Agent creado")

## 9. Crear Agente Summarizer

Crea un reporte final del proceso.

In [ ]:
summarizer = client.create_agent(
    name="Summarizer",
    instructions=(
        "You are a summarizer agent. "
        "Create a final publication report that includes:\n"
        "1. A brief summary of the published content\n"
        "2. The workflow path taken (direct approval or edited)\n"
        "3. Key highlights and takeaways\n"
        "Keep it concise and professional."
    ),
)

print("✅ Summarizer Agent creado")

## 10. Construir el Workflow con Ramificación

Este es el corazón del sistema. Definimos:
- El agente inicial (writer)
- Las conexiones entre agentes
- Las condiciones de ramificación
- El punto de convergencia (summarizer)

In [ ]:
workflow = (
    WorkflowBuilder(
        name="Content Review Workflow",
        description="Multi-agent content creation workflow with quality-based routing",
    )
    .set_start_executor(writer)
    .add_edge(writer, reviewer)
    # Branch 1: High quality (>= 80) goes directly to publisher
    .add_edge(reviewer, publisher, condition=is_approved)
    # Branch 2: Low quality (< 80) goes to editor first, then publisher
    .add_edge(reviewer, editor, condition=needs_editing)
    .add_edge(editor, publisher)
    # Both paths converge: Publisher → Summarizer
    .add_edge(publisher, summarizer)
    .build()
)

print("✅ Workflow construido con ramificación y convergencia")
print("\n📋 Flujo:")
print("   Writer → Reviewer → [branch]")
print("     ├─ (score >= 80) → Publisher → Summarizer")
print("     └─ (score < 80)  → Editor → Publisher → Summarizer")

## 11. Visualizar el Workflow (UI)

El framework incluye una UI de desarrollo para visualizar y probar el workflow.

**Nota**: En un notebook, esto abrirá una nueva pestaña del navegador.

In [ ]:
from agent_framework.devui import serve

# Esto iniciará un servidor local y abrirá el navegador
# Descomenta la siguiente línea para ejecutar
# serve(entities=[workflow], port=8093, auto_open=True)

print("💡 Descomenta la línea anterior para iniciar la UI de desarrollo")
print("   La UI te permite:")
print("   - Visualizar el workflow gráficamente")
print("   - Ejecutar el workflow con diferentes inputs")
print("   - Ver el flujo de datos entre agentes")
print("   - Depurar cada paso del proceso")

## Conclusión

Este ejemplo avanzado demuestra:

### 1. **Workflow Complejo**
- Múltiples agentes especializados
- Ramificación basada en condiciones
- Convergencia de caminos
- Flujo de datos estructurado

### 2. **Salida Estructurada**
- Uso de Pydantic para `response_format`
- Garantiza formato consistente
- Facilita la lógica de decisión

### 3. **Ramificación Condicional**
- Las funciones de condición determinan el camino
- Permite flujos adaptativos
- Optimiza el proceso según calidad

### 4. **Convergencia**
- Múltiples caminos llevan al mismo destino
- El summarizer recibe el resultado final independientemente del camino
- Mantiene consistencia en la salida

### Aplicaciones Prácticas:
- **Content pipelines**: Blogs, artículos, documentación
- **Code review workflows**: Revisión automática de código
- **Quality assurance**: Testing y validación automática
- **Approval processes**: Flujos de aprobación multi-nivel
- **Data processing**: ETL con validación y corrección

### Ventajas del Patrón:
- **Escalable**: Fácil agregar más agentes o condiciones
- **Mantenible**: Cada agente tiene responsabilidad única
- **Flexible**: Condiciones dinámicas permiten adaptación
- **Trazable**: El flujo es explícito y visualizable

Este patrón es ideal cuando necesitas procesos de múltiples pasos con validación y corrección automática.